Tomi Salomaa <br>
Student number  <br>
February, 9, 2021  <br>

# Exercise 2 | TKO_2096 Application of Data Analysis 2021

#### Prediction of the metal ion content from multi-parameter data <br>
- Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), for each sample using number of neighbors k = 3.<br> <br>

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score.
    - Implement your own Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement your own Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Return your solution as a Jupyter Notebook file (include your full name in the file name).
    - Submit to moodle your solution on ** Wednesday 10 of February** at the latest.

## Import libraries

In [1]:
#In this cell import all libraries you need. 
import numpy as np
import pandas as pd
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from scipy import stats
from google.colab import files

In [2]:
#!pip install -U scikit-learn

In [3]:
print('Scikit-learn: {}'.format(sklearn.__version__))

Scikit-learn: 0.24.1


In [4]:
# upload dataset source file for google colab IDE
#uploaded = files.upload()

## Read and visualize the dataset

In [5]:
# Reading the dataset
water_data = pd.read_csv('Water_data.csv')

# Presenting shape of the dataset
print("Shape of 'water_data' dataset:",water_data.shape)

# Printing the first 5 rows
print('\nFirst 5 rows of the dataset')
print('-'*45)
print(water_data.head())

Shape of 'water_data' dataset: (225, 6)

First 5 rows of the dataset
---------------------------------------------
   c_total     Cd      Pb    Mod1  Mod2    Mod3
0     2000  800.0  1200.0  126430  2604    6996
1       35   14.0    21.0   20597   271  138677
2       35   14.0    21.0   24566   269  161573
3       35   35.0     0.0  105732   971  132590
4      100   20.0    80.0   57774  5416   93798


#### To show understanding of the data, answer the following questions:
- How many different mixtures of Cadmium (Cd) and Lead (Pb) were measured? <br>
- How many total concentrations (c_total) were measured? <br>
- How many mixtures have less than 4 replicas? <br>
- How many mixtures have 4 or more replicas? Print out c_total, Cd and Pb for those concentrations.<br>

In [6]:
#In this cell write the code to answer the previous questions and print the answers.

# Finding out the amount of unique mixtures of Cd and Pb measured
cd_and_pb_col = water_data[['c_total','Cd','Pb']]
duplicates_mixtures = cd_and_pb_col.groupby(cd_and_pb_col.columns
                                            .tolist(),as_index=False).size()
all_dup_rows_mix = duplicates_mixtures['size'].sum()
print('Amount of mixtures in the dataset')
print('-'*35)
print('Amount of unique Cd and Pb mixture combinations:',
      len(duplicates_mixtures.index))
print('Amount of rows with all duplicates combined:',all_dup_rows_mix)
if all_dup_rows_mix < len(water_data):
  print('Unique combinations and their duplicates amount to',all_dup_rows_mix)
  print('which is less than the total of',len(water_data),
        'rows observed in the original dataset.')
  print('Thus, the amount of unique mixtures in the dataset amounts to:',
        len(duplicates_mixtures.index) + len(water_data) - all_dup_rows_mix)
  print('(Unique mixtures without duplicates:',
        len(water_data) - all_dup_rows_mix,')')
elif all_dup_rows_mix == len(water_data):
  print('Unique combinations and their duplicates fill the original dataset exactly.')
  print('Thus, the amount of observed uniques (',len(duplicates_mixtures.index),
        ') is the amount of different mixtures.')
else:
  print('This cannot be!')

# Finding out the amount of total concentrations measured
print('\n')
has_null_values = water_data['c_total'].isnull().values.any()
if has_null_values:
  print('Null c_total values:',has_null_values)
  print('Null values in total:',water_data['your column name'].isnull().sum())
else:
  c_total_col = water_data[['c_total']]
  duplicates_c_tot = c_total_col.groupby(c_total_col.columns
                                   .tolist(),as_index=False).size()
  c_total_zero = duplicates_c_tot.loc[duplicates_c_tot['c_total'] == 0]
  all_dup_rows_c_tot = duplicates_c_tot['size'].sum()
  print('Total concentrations measured')
  print('-'*30)
  print('The dataset does not contain null c_total values.')
  print('     Total concentration value greater than 0:',
        all_dup_rows_c_tot-c_total_zero['size'][0])
  print('     Total concentration value 0:',c_total_zero['size'][0])
  print('As mentioned in the "Metal ion concentration data" lecture by Perez,',
        '\na value of 0 can be assumed to be a result indicating pure tap water.',
        '\nThus, the measured amount of total concentrations is',
        all_dup_rows_c_tot)

print('\n')
print('More info on duplicate data')
print('-'*30)

# Finding out mixtures with less than 4 duplicates
less_than_four_duplicates = 0
for i in range(len(duplicates_mixtures)):
  if duplicates_mixtures.iloc[i]['size'] < 4:
    less_than_four_duplicates += 1
print('Mixtures with less than 4 duplicates:',less_than_four_duplicates)
print('Mixtures with 4 or more duplicates:',
      len(duplicates_mixtures.index)-less_than_four_duplicates)

# Presenting mixtures with 4 or more duplicates
# (c_total, Cd and Pb for those concentrations)
new_dup_mix = duplicates_mixtures.rename({'size':'duplicates'},axis=1)
print('\n')
print('Mixtures with 4 or more duplicates are:\n')
print(new_dup_mix.loc[new_dup_mix['duplicates']>=4]
      .to_string(index=False))

Amount of mixtures in the dataset
-----------------------------------
Amount of unique Cd and Pb mixture combinations: 67
Amount of rows with all duplicates combined: 225
Unique combinations and their duplicates fill the original dataset exactly.
Thus, the amount of observed uniques ( 67 ) is the amount of different mixtures.


Total concentrations measured
------------------------------
The dataset does not contain null c_total values.
     Total concentration value greater than 0: 222
     Total concentration value 0: 3
As mentioned in the "Metal ion concentration data" lecture by Perez, 
a value of 0 can be assumed to be a result indicating pure tap water. 
Thus, the measured amount of total concentrations is 225


More info on duplicate data
------------------------------
Mixtures with less than 4 duplicates: 43
Mixtures with 4 or more duplicates: 24


Mixtures with 4 or more duplicates are:

 c_total     Cd     Pb  duplicates
      50    0.0   50.0           4
      50   10.0   40

## Standardization of the dataset

In [7]:
#Standardize the dataset features by removing the mean and scaling to unit variance. 
#In other words, use z-score to scale the dataset features (Mod1, Mod2, Mod3)
def df_z_standardizer(df,method=False):
  
  standardized_df = df.copy()
  if not method:
    # Go through columns and transform the values by 
    # removing mean and divide with standard variance.
    for column in standardized_df.columns:
        standardized_df[column] = (standardized_df[column] -
                                   standardized_df[column].mean()) / standardized_df[column].std()
  elif method == 'stats':
    scaled = stats.zscore(df)
    #scaler = StandardScaler()
    #scaled = scaler.fit_transform(standardized_df)
    standardized_df = pd.DataFrame(scaled, columns=['Mod1','Mod2','Mod3'])

  elif method == 'scaler':
    scaler = StandardScaler()
    scaled = scaler.fit_transform(standardized_df)
    standardized_df = pd.DataFrame(scaled, columns=['Mod1','Mod2','Mod3'])
    
  return standardized_df

# Build a new dataframe with standardized feature values
water_data_features = water_data[['Mod1','Mod2','Mod3']]
z_std_features = df_z_standardizer(water_data_features,'scaler')

standardized_water_data = water_data.drop(columns=['Mod1','Mod2','Mod3'])
standardized_water_data = pd.concat([standardized_water_data,z_std_features],
                                    axis=1)

# Creating datasets to use in evaluations
water_data_x_y = standardized_water_data
# Print the 5 first samples (i.e. rows) of the scaled dataset
print(standardized_water_data.head())

   c_total     Cd      Pb      Mod1      Mod2      Mod3
0     2000  800.0  1200.0  0.166505 -0.508756 -1.499041
1       35   14.0    21.0 -0.892616 -0.701641  0.685861
2       35   14.0    21.0 -0.852896 -0.701806  1.065760
3       35   35.0     0.0 -0.040629 -0.643767  0.584863
4      100   20.0    80.0 -0.520568 -0.276268 -0.058789


## C-index code 

In [8]:
def cindex(true, pred, regression=False,silent=False):
  n = 0
  n_total = 0
  pairs = list()
  neg_pairs = list()
  pos_pairs = list()

  # Create true,prediction -pairs
  for i in range(len(true)):
    pairs.append([true[i],pred[i]])
  
  if not regression:
    # this only works with "binary"
    for i in range(len(pairs)):
      if pairs[i][0] != 1:
        neg_pairs.append(pairs[i])
      else:
        pos_pairs.append(pairs[i])
    for i in range(len(neg_pairs)):
      for j in range(len(pos_pairs)):
        test_one = neg_pairs[i][0]>pos_pairs[j][0]
        test_two = neg_pairs[i][1]>pos_pairs[j][1]
        test_three = neg_pairs[i][1]==pos_pairs[j][1]

        if test_one == test_two:
          n += 1
          if test_three:
            n -= 0.5
        n_total += 1
  else:
    # this should apply to regression cases
    for i in range(len(pairs)):
      for j in range(i+1,len(pairs)):
        if pairs[i][0] != pairs[j][0]:
          test_one = pairs[i][0]<pairs[j][0]
          test_two = pairs[i][1]<pairs[j][1]
          test_three = pairs[i][1]==pairs[j][1]
          n_total += 1
          if test_one == test_two:
            n += 1
            if test_three:
              n -= 0.5
  cindx = n / n_total

  if not silent:
    print('CONCORDANCE INDEX RESULTS')
    print('-'*35)
    print('N:',n)
    print('N total:',n_total)
    print('C-INDEX:',cindx)
  
  return cindx

In [9]:
#test cindex function with following values
true_labels = [-1, 1, 1, -1, 1]
predictions = [0.60, 0.80, 0.75, 0.75, 0.70]
cindx = cindex(true_labels, predictions)

CONCORDANCE INDEX RESULTS
-----------------------------------
N: 4.5
N total: 6
C-INDEX: 0.75


In [10]:
# Testing with the values from C-index in regression case example (Concordance_index_in_general_case.ppt)
true_data_points = [7.1,2.3,5.4,1.8]
pred_data_points = [7.4,3.4,8.5,3.9]
cindex(true_data_points,pred_data_points,True)

CONCORDANCE INDEX RESULTS
-----------------------------------
N: 4
N total: 6
C-INDEX: 0.6666666666666666


0.6666666666666666

## Functions

In [11]:
# Leave-One-Out CV
def LOO_cross_validation(dataset, neighbours_n = 3):
  folds = list(dataset)
  cindex_ctotal_arr = list()
  cindex_cd_arr = list()
  cindex_pb_arr = list()
  predictions = list()
  true_labels = list()
  knn = KNeighborsRegressor(n_neighbors=neighbours_n)

  # Splitting folds and data -- in this case iterating the dataset
  for i in range(len(folds)):
    # Fit with all but the index
    train_set = list(folds)
    test_set = train_set.pop(i)
    y_train = [arr[:3] for arr in train_set]
    x_train = [arr[3:] for arr in train_set]


    y_train_nparr = np.asarray(y_train)
    x_train_nparr = np.asarray(x_train)

    knn.fit(x_train, y_train_nparr)
    # Test with the index
    y_test = np.array([test_set[0],test_set[1],test_set[2]])
    x_test = np.array([test_set[3],test_set[4],test_set[5]])
    # Gather predictions and true labels
    pred = knn.predict(x_test.reshape(1,-1))
    
    predictions.append(pred[0])
    true_labels.append(y_test)

  temp_ctotal_arr = list()
  temp_ctotal_pred_arr = list()
  temp_cd_arr = list()
  temp_cd_pred_arr = list()
  temp_pb_arr = list()
  temp_pb_pred_arr = list()
  
  for i in range(len(true_labels)):
    temp_ctotal_arr.append(true_labels[i][0])
    temp_ctotal_pred_arr.append(predictions[i][0])
    temp_cd_arr.append(true_labels[i][1])
    temp_cd_pred_arr.append(predictions[i][1])
    temp_pb_arr.append(true_labels[i][2])
    temp_pb_pred_arr.append(predictions[i][2])
  
  cindex_ctotal_arr.append(temp_ctotal_arr)
  cindex_ctotal_arr.append(temp_ctotal_pred_arr)

  cindex_cd_arr.append(temp_cd_arr)
  cindex_cd_arr.append(temp_cd_pred_arr)

  cindex_pb_arr.append(temp_pb_arr)
  cindex_pb_arr.append(temp_pb_pred_arr)

  cindex_ctotal = cindex(cindex_ctotal_arr[0],cindex_ctotal_arr[1],True,True)
  cindex_cd = cindex(cindex_cd_arr[0],cindex_cd_arr[1],True,True)
  cindex_pb = cindex(cindex_pb_arr[0],cindex_pb_arr[1],True,True)

  print('LEAVE-ONE-OUT CROSS-VALIDATION RESULTS')
  print('-'*42)
  print('C-index results:')
  print('   c_total:',cindex_ctotal)
  print('   cd:',cindex_cd)
  print('   pb:',cindex_pb)
  print('-'*42)

# Leave-Replicas-Out split
def LRO_split(dataset):
  dataset_split = list()
  temp_list = list()
  dataset_copy = dataset
  # Sort the dataset based on Cd and Pb columns so the groups are sorted as a result
  grouped_data = dataset_copy.sort_values(['Cd','Pb'],
                                          ascending=[True, False]).values
  # Go through the sorted data and form the splits based on groups
  for i in range(len(grouped_data)):
    if not temp_list:
      temp_list.append(grouped_data[i])
      continue
    if temp_list[0][1] == grouped_data[i][1] and temp_list[0][2] == grouped_data[i][2]:
      temp_list.append(grouped_data[i])
    else:
      dataset_split.append(temp_list)
      temp_list = list()
      temp_list.append(grouped_data[i])
  dataset_split.append(temp_list)

  return dataset_split

# Leave-Replicas-Out CV
def LRO_cross_validation(dataset, neighbours_n = 3):
  # Form folds by splitting the data with LRO_split()
  folds = LRO_split(dataset)
  cindex_arr = list()
  knn = KNeighborsRegressor(n_neighbors = neighbours_n)
  true_labels = list()
  predictions = list()
  cindex_ctotal_arr = list()
  cindex_cd_arr = list()
  cindex_pb_arr = list()

  # Iterate folds to perform the train and test split group by group
  for i in range(len(folds)):
    train_set_temp = list(folds)
    train_set = list()
    test_set = train_set_temp.pop(i)
    for j in range(len(train_set_temp)):
      for k in range(len(train_set_temp[j])):
        train_set.append(train_set_temp[j][k])
    y_train = [arr[:3] for arr in train_set]
    x_train = [arr[3:] for arr in train_set]
    knn.fit(x_train, y_train)

    # Test with the index group
    prediction_mean = list()
    for j in range(len(test_set)):
      y_test = np.array([test_set[j][0],test_set[j][1],test_set[j][2]])
      x_test = np.array([test_set[j][3],test_set[j][4],test_set[j][5]])
      
      # Gather predictions and true labels
      pred = knn.predict(x_test.reshape(1,-1))
      predictions.append(pred[0])
      true_labels.append(np.array([test_set[0][0],test_set[0][1],test_set[0][2]]))

  temp_ctotal_arr = list()
  temp_ctotal_pred_arr = list()
  temp_cd_arr = list()
  temp_cd_pred_arr = list()
  temp_pb_arr = list()
  temp_pb_pred_arr = list()
  
  for i in range(len(true_labels)):
    temp_ctotal_arr.append(true_labels[i][0])
    temp_ctotal_pred_arr.append(predictions[i][0])
    temp_cd_arr.append(true_labels[i][1])
    temp_cd_pred_arr.append(predictions[i][1])
    temp_pb_arr.append(true_labels[i][2])
    temp_pb_pred_arr.append(predictions[i][2])
  
  cindex_ctotal_arr.append(temp_ctotal_arr)
  cindex_ctotal_arr.append(temp_ctotal_pred_arr)

  cindex_cd_arr.append(temp_cd_arr)
  cindex_cd_arr.append(temp_cd_pred_arr)

  cindex_pb_arr.append(temp_pb_arr)
  cindex_pb_arr.append(temp_pb_pred_arr)

  cindex_ctotal = cindex(cindex_ctotal_arr[0],cindex_ctotal_arr[1],True,True)
  cindex_cd = cindex(cindex_cd_arr[0],cindex_cd_arr[1],True,True)
  cindex_pb = cindex(cindex_pb_arr[0],cindex_pb_arr[1],True,True)

  print('LEAVE-REPLICAS-OUT CROSS-VALIDATION RESULTS')
  print('-'*42)
  print('C-index results:')
  print('   c_total:',cindex_ctotal)
  print('   cd:',cindex_cd)
  print('   pb:',cindex_pb)
  print('-'*42)

## Results for Leave-One-Out cross-validation

In [12]:
#In this cell run your code for leave-One-Out cross-validation and print the corresponding results.
LOO_cross_validation(standardized_water_data.values)

LEAVE-ONE-OUT CROSS-VALIDATION RESULTS
------------------------------------------
C-index results:
   c_total: 0.911953783337677
   cd: 0.8983588758508373
   pb: 0.8731782686766609
------------------------------------------


## Results for Leave-Replicas-Out cross-validation

In [13]:
#In this cell run your script for leave-Replicas-Out cross-validation and print the corresponding results.
LRO_cross_validation(standardized_water_data)

LEAVE-REPLICAS-OUT CROSS-VALIDATION RESULTS
------------------------------------------
C-index results:
   c_total: 0.814025714533924
   cd: 0.7578402305090408
   pb: 0.7675491710861486
------------------------------------------


## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation approach had more optimistic results?
- Which cross-validation generalize better on unseen data? Why?

#In this cell write your answers to the questions about Interpretation of Results.

<p>Regarding <br/>
<b>C total:</b> LOOCV was more optimistic with a cindex of ~0.912 vs LROCV with ~0.814<br/>
<b>Cd:</b> LOOCV was more optimistic with a cindex of ~0.898 vs LROCV with ~0.758<br/>
<b>Pb:</b> LOOCV was more optimistic with a cindex of ~0.873 vs LROCV with ~0.768<br/></p>
<p>Overall the LOOCV seems to be the more optimistic one.</p>

<p>Most likely the LROCV is the one to better generalize itself on unseen data. Scoring-wise LOOCV may benefit from replicates in the data which causes data leakage between test and training sets and thus one could argue that in this case it would be - if not absolutely correct then at least more realistic with the given results - to assume that the LROCV scoring tells the truth about its generalizing ability moreso than LOOCV.</p>